In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d81726ed111e9f765750c22ab4c67f37cd85af7515ec8023fd57d2a2cf30c6e6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [6]:
spark = SparkSession.builder.master("local[*]").getOrCreate() #Seteaza numarul de core-uri de utilizat (* = toate)

csv_path = "/content/Students/StudentsPerformance.csv" #Locatia datelor noastre

df = spark.read.csv(csv_path, header=True, inferSchema=True) #Incarcarea datelor din CSV intr-un dataframe Spark

In [7]:
table_name = "my_table" #definirea unei variabile table_name
df.createOrReplaceTempView(table_name) #crearea unui view temporar
query = "SELECT * FROM my_table" #definirea unei interogari SQL
result = spark.sql(query) #rularea interogarii
result.show() #afisarea rezultatelor

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [9]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [13]:
df.describe().show()


+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null|        null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null|        null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          